# **Gemma を使ったファインチューニング入門**
本ハンズオンでは、[日本語版 Gemma 2 2B](https://developers-jp.googleblog.com/2024/10/gemma-2-for-japan.html) (`gemma-2-2b-jpn`) と [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index) 及び [Hugging Face TRL](https://huggingface.co/docs/trl/en/index) を使って、[LoRA](https://arxiv.org/abs/2106.09685) によるファインチューニングを行います。

前提条件: 以下の作業が完了していること
* Hugging Face アカウントの作成
* Hugging Face Access Token の発行
* Gemma の利用規約への同意

推奨ランタイム:
*    g2-standard-12 (NVIDIA L4 * 1) もしくは a2-highgpu-1g (NVIDIA A100 * 1)






# **1. 環境準備**

In [ ]:
# @title 1-1. 必要パッケージをインストール
# @markdown [補足] Hugging Face 関連

# @markdown - `transformers` は、生成 AI や自然言語処理 (NLP) タスク用のモデルを簡単にダウンロード、トレーニング、および使用できるようにするパッケージ

# @markdown - `accelerate` はモデルのトレーニングや推論を、CPU、GPU、TPUなど多様なデバイスや分散環境で効率的に実行するためのパッケージ

# @markdown - `datasets` は Hugging Face で公開されている様々なデータセットを簡単にダウンロード、前処理、操作できるパッケージ

# @markdown - `peft` その名の通り PEFT を行うためのパッケージ

# @markdown - `trl` トランスフォーマーモデルを SFT や RLHF でトレーニングするためのフルスタック パッケージ

# @markdown [補足] PyTorch 関連

# @markdown - `torch` (PyTorch) は 深層学習フレームワークで、Hugging Face Transformers のバックエンドとして GPU や LoRA の重み更新などを実際にサポートしているパッケージ

# @markdown - `tensorboard` は、ファインチューニング中に出力されるログを可視化するパッケージ

# Pytorch 関連のパッケージをインストール
!pip install --upgrade torch tensorboard

# Hugging Face 関連のパッケージをインストール
!pip install --upgrade \
  transformers \
  accelerate \
  datasets \
  peft \
  trl

In [ ]:
# @title 1-2. Hugging Face の Access Token を変数に代入
# @markdown [補足]

# @markdown - Hugging Face から Gemma モデルをダウンロードする時や、ファインチューニングしたモデルを Hugging Face にプッシュする時に使います
hf_access_token = "" # @param {"type":"string"}

In [ ]:
# @title 1-3. Hugging Face にログイン
from huggingface_hub import login

login(
  token=hf_access_token,
  add_to_git_credential=False
)

# **2. `gemma-2-2b-int-jp` を使用**

In [ ]:
# @title 2-1. トークナイザーとモデルの読み込み
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Hugging Face 上で有効なモデルの ID を変数に代入
model_id = "google/gemma-2-2b-jpn-it"

# トークナイザーとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
# @title 2-2. プロンプトの作成と推論、結果確認
# Gemma に聞く質問を定義
messages = [
    {"role": "user", "content": "日本の首都はどこですか？"},
]

# Gemma への入力用に Messages をトークン化
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

# Gemma による生成と返ってきたトークンを自然言語にデコード
outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
gemma_response1 = generated_text.strip()

print(gemma_response1)

In [ ]:
# @title 2-3. メモリを開放
del model, tokenizer

# **3. ファインチューニング用のデータセットを準備**

In [ ]:
# @markdown [補足]

# @markdown - 本デモでは以下のデータセットを使用しています：
# @markdown - データセット名: bbz662bbz/databricks-dolly-15k-ja-gozaru
# @markdown - データセットの提供元: bbz662bbz
# @markdown - ライセンス: CC BY-SA 3.0
# @markdown - https://huggingface.co/datasets/bbz662bbz/databricks-dolly-15k-ja-gozaru

from datasets import load_dataset

# データセットの読み込み
dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozaru", split="train")
dataset = dataset.filter(lambda example: example["category"] == "open_qa")

# データセットをプロンプト形式に変換する関数の定義
def generate_prompt(example):
    return """<bos><start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<eos>""".format(example["instruction"], example["output"])

# textカラムの追加
def add_text(example):
    example["text"] = generate_prompt(example)
    return example
dataset = dataset.map(add_text)
dataset = dataset.remove_columns(["input", "category", "output", "index", "instruction"])

# データセットの分割
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# 最終的なデータセットの確認
print(len(train_dataset))
print(train_dataset[0])

## **4. `TRL` と `SFTTrainer` を使ったトレーニグ**

In [ ]:
# @title 4-1. ファインチューニング用にトークナイザーとモデルの読み込み
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# from trl import setup_chat_format

# Hugging Face 上で有効なモデルの ID を指定
model_id = "google/gemma-2-2b-jpn-it"

# トークナイザーとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# 警告を回避するためトークナイザーのパdディング方向を右側に設定
tokenizer.padding_side = 'right'

In [ ]:
# @title 4-2. LoRA の設定
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=32,         # LoRAのスケーリング係数（大きいほど更新の影響が大きくなる）
        lora_dropout=0.1,      # ドロップアウト率、LoRAのパラメータ更新時の過学習を防ぐ
        r=8,                   # LoRAでのランク（低ランク行列の次元）、モデルの容量と精度のバランスに影響
        bias="none",           # バイアスの学習方法（noneはバイアスを更新しない設定）
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # LoRAを適用する対象モジュール（プロジェクション層や特定のネットワーク層）
        task_type="CAUSAL_LM", # タスクの種類、ここでは自己回帰型言語モデル (Causal Language Modeling) を指定
)

In [ ]:
# @title 4-3. ハイパーパラメータの設定
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="gemma-2-2b-int-jpn-lora1",  # モデルを保存するディレクトリとリポジトリID
    num_train_epochs=3,                     # 学習エポック数
    per_device_train_batch_size=1,          # デバイスごとの学習バッチサイズ
    gradient_accumulation_steps=2,          # バックワード/更新を行う前のステップ数
    gradient_checkpointing=True,            # メモリ節約のために勾配チェックポイントを使用
    optim="adamw_torch_fused",              # 融合AdamWオプティマイザを使用
    logging_steps=10,                       # 10ステップごとにログ出力
    save_strategy="epoch",                  # 各エポック終了時にチェックポイントを保存
    learning_rate=2e-4,                     # 学習率（QLoRA論文に基づく）
    bf16=True,                              # 対応GPUがある場合にbfloat16精度を使用
    tf32=True,                              # 対応GPUがある場合にtf32精度を使用
    max_grad_norm=0.3,                      # 最大勾配ノルム（QLoRA論文に基づく）
    warmup_ratio=0.03,                      # ウォームアップ比率（QLoRA論文に基づく）
    lr_scheduler_type="constant",           # 定数学習率スケジューラを使用
    push_to_hub=True,                       # モデルをHugging Face Hubにプッシュ
    report_to="tensorboard",                # メトリクスをtensorboardにレポート
)

In [ ]:
# @title 4-4. SFTTrainer クラスのインスタンスを作成
from trl import SFTTrainer

# モデルに入力できるテキストの最大長を 1024トークンに制限
max_seq_length = 1024

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
     dataset_text_field="text",
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

In [ ]:
# @title 4-5. トレーニングの実行
from datetime import datetime

# トレーニング開始前に現在時刻を出力
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Training started at: {current_time}")

# トレーニングを実行開始
trainer.train()

# トレーニングが完了したモデルを保存
trainer.save_model()

In [ ]:
# @title 4-6. メモリの開放
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
# @title 4-7. LoRA adapter とオリジナルモデルをマージ
from peft import AutoPeftModelForCausalLM

# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    # low_cpu_mem_usage=True,
    device_map="auto"
)
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(args.output_dir,safe_serialization=True, max_shard_size="2GB")

## **5. ファインチューニング後のモデルをテスト**


In [ ]:
# @title 5-1. トークナイザーとモデルの読み込み
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = args.output_dir

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

In [ ]:
# @title 5-2. プロンプトの作成と推論、結果確認
messages = [
    {"role": "user", "content": "日本の首都はどこですか？" },
]

inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
gemma_response2 = generated_text.strip()

print(gemma_response2)